# Request data from USGS and save to CSV

In [ ]:
import requests
import pandas as pd

# Set the API endpoint URL
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

# Define the bounding box for the area of interest
min_latitude = 10
max_latitude = 60
min_longitude = 134 #117 is wide
max_longitude = 174 #165 more tight

# Create an empty list to hold the earthquake data
earthquakes = []

for year in range(1973, 2023):
    for month in range(1, 13):
        # Set the parameters for the API request
        starttime = f'{year}-{month:02d}-01'
        endtime = f'{year}-{month+1:02d}-01'
        if month == 12:
            endtime = f'{year+1}-01-01'
        params = {
            'format': 'geojson',
            'starttime': starttime,
            'endtime': endtime,
            'minmagnitude': '0',
            'maxmagnitude': '10',
            'minlatitude': min_latitude,
            'maxlatitude': max_latitude,
            'minlongitude': min_longitude,
            'maxlongitude': max_longitude,
            'mindepth': '0',
            'maxdepth': '1000',
        }

        # Send the API request and get the response
        response = requests.get(url, params=params)

        # Parse the JSON response
        data = response.json()

        # Extract the data for each earthquake and append it to the list
        for feature in data['features']:
            longitude = feature['geometry']['coordinates'][0]
            latitude = feature['geometry']['coordinates'][1]
            time = pd.to_datetime(feature['properties']['time'], unit='ms')
            magnitude = feature['properties']['mag']
            depth = feature['geometry']['coordinates'][2]
            focal_mechanism = feature['properties']['types'][0]
            earthquake = {'Longitude': longitude, 'Latitude': latitude, 'Time': time, 'Magnitude': magnitude, 'Depth': depth}
            earthquakes.append(earthquake)

# Create a DataFrame from the list of earthquake data
df = pd.DataFrame(earthquakes)

# Cut off magnitudes of 0; delete outliers
df = df[df.Magnitude > 0]

# save as csv. PLEASE DON'T RUN THIS, THANKS!
# df.to_csv('data/Japan_10_60_134_174_1973_2023_V2.csv')

# Print the first few rows of the DataFrame
print(df.head())
print(df.shape, df.Magnitude.min())
